# Final Report

In [132]:
from scheduler import *
import matplotlib.pyplot as plt

In [133]:
import sys
sys.path.append('src')

from scheduler.linkedeventlist import LinkedEventList
from scheduler.event import EventNode

In [134]:
def create_event_node(title: str, time: str = "10:00"):
    return EventNode(
        title=title,
        date="2025-10-13",
        time=time,
        location="Test"
    )

In [135]:
test_list = LinkedEventList()

test_node1 = create_event_node(title = "Event 1", time = "11:00")
test_list.insert(test_node1)

t_node = test_list.list_all(sort = False)
for e in t_node:
    print(e.time)

11:00


In [136]:
test_node2 = create_event_node(title= "Event 2", time = "08:00")

test_list.insert(test_node2)

t_node = test_list.list_all(sort = False)
for e in t_node:
    print(e.time, e.title)

11:00 Event 1
08:00 Event 2


In [137]:
test_node3 = create_event_node("Event 3")
test_list.insert(test_node3)

In [138]:
t_node = test_list.list_all(sort = True)
for e in t_node:
    print(e.time, e.title)

08:00 Event 2
10:00 Event 3
11:00 Event 1


In [139]:
print(test_node1.title)

Event 2


In [140]:
test_list.delete(event = test_node1)

block1 Event 2


In [141]:
t_node = test_list.list_all(sort = False)
for e in t_node:
    print(e.time)

10:00
11:00


Final Report: Campus Event Scheduling System